# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `production_2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [2]:
# Write your code below.
%load_ext dotenv
%dotenv 


In [3]:
import dask
dask.config.set({'dataframe.query-planning': True})
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [4]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv('PRICE_DATA')
parquet_files = glob(os.path.join(PRICE_DATA, "*/*.parquet"))
dd_px = dd.read_parquet(PRICE_DATA).set_index("ticker")

print(dd_px.columns)

Index(['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume', 'sector',
       'subsector', 'year'],
      dtype='object')


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [5]:
# Write your code below.
dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1), Adj_close_lag = x['Adj Close'].shift(1))
).assign(
    returns = lambda x: x['Adj Close']/x['Adj_close_lag'] - 1
).assign(
    hi_lo_range = lambda x: (x['High']-x['Low'])
))


/tmp/ipykernel_112420/226744910.py:2: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [6]:
# Write your code below.
pd_feat = dd_feat.compute()


In [7]:
pd_feat

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_close_lag,returns,hi_lo_range
ticker,,,,,,,,,,,,,,
OXY,2002-01-02,12.670362,12.742217,12.440428,12.699104,6.993292,2083374,Energy,Oil & Gas Exploration & Production,2002,NaN,NaN,NaN,0.301789
OXY,2002-01-03,12.794910,12.794910,12.550605,12.641620,6.961636,2154768,Energy,Oil & Gas Exploration & Production,2002,12.699104,6.993292,-0.004527,0.244306
OXY,2002-01-04,12.584137,12.770959,12.550605,12.747007,7.019676,1297412,Energy,Oil & Gas Exploration & Production,2002,12.641620,6.961636,0.008337,0.220354
OXY,2002-01-07,12.670362,12.842813,12.560185,12.703894,6.995933,1525581,Energy,Oil & Gas Exploration & Production,2002,12.747007,7.019676,-0.003382,0.282627
OXY,2002-01-08,12.713475,12.713475,12.478750,12.512282,6.890413,1186354,Energy,Oil & Gas Exploration & Production,2002,12.703894,6.995933,-0.015083,0.234725
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
INTU,2006-12-22,30.730000,30.730000,30.110001,30.299999,27.002762,1790700,Information Technology,Application Software,2006,30.639999,27.305763,-0.011097,0.619999
INTU,2006-12-26,30.170000,30.360001,30.020000,30.320000,27.020580,1594600,Information Technology,Application Software,2006,30.299999,27.002762,0.000660,0.340000
INTU,2006-12-27,30.730000,30.990000,30.520000,30.780001,27.430523,1486800,Information Technology,Application Software,2006,30.320000,27.020580,0.015171,0.469999


In [11]:
pd_feat_window = pd_feat["returns"].rolling(10).mean()
pd_feat_window

ticker
OXY          NaN
OXY          NaN
OXY          NaN
OXY          NaN
OXY          NaN
          ...   
INTU   -0.001740
INTU   -0.001674
INTU    0.000686
INTU   -0.001857
INTU   -0.002980
Name: returns, Length: 2746333, dtype: float64

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?

*My Answer*: No, it's not necessary, because Dask dataframes also has rolling().mean function. 

+ Would it have been better to do it in Dask? Why?

*My Answer*: Dask performs the lazy execution, and execute the functions in the moment when you compute the dataframe to pandas dataframe. So it won't be better since the computation time and results would be the same. 

(1 pt)

## Criteria

|Criteria|Complete|Incomplete|
|---------------------|----|----|
|Calculations         |Calculations were done correctly.|Calculations were not done correctly.|
|Explanation of answer|Answer was concise and explained the learner's reasoning in depth.|Answer was not concise and did not explained the learner's reasoning in depth.|

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.